In [3]:
import shutil
import os
import torch
from rundenoise import run_denoise, run_denoise_multi
from runSR import runSR
import argparse
from PIL import Image
import io
import base64
import uuid
import requests

def denoise_url(url):
    response = requests.get(url, timeout=10)
    response.raise_for_status()

    # Mở ảnh từ dữ liệu tải về
    image = Image.open(io.BytesIO(response.content))
    ext = image.format.lower()
    if ext not in ['jpeg', 'jpg', 'png', 'webp', 'bmp', 'tiff']:
        # return {"error": "Unsupported image format"}
        # raise HTTPException(status_code=415, detail="Unsupported image format")
        print("erorr!!!")
    if image.mode != 'RGB':
        image = image.convert('RGB')
        
    session_id = str(uuid.uuid4())
    up_dir = f'/home/duongnhan/Chon/Capstone_project/Backend/uploads/up/{session_id}/'
    save_dir = f'/home/duongnhan/Chon/Capstone_project/Backend/uploads/result/{session_id}/'
    os.makedirs(os.path.join(up_dir, 'input'), exist_ok=True)
    os.makedirs(os.path.join(up_dir, 'gt'), exist_ok=True)
    # os.makedirs(save_dir, exist_ok=True)
    filename = f"{uuid.uuid4().hex}.png"
    image_input_path = os.path.join(up_dir,"input", filename)
    image_gt_path = os.path.join(up_dir,"gt", filename)
    image.save(image_input_path)
    image.save(image_gt_path)

    with open(os.path.join(up_dir, "CT.txt"), "w", encoding="utf-8") as f:
        f.write(f"/input/{filename}\n")
    textfile = '/CT.txt'
    pred_image = run_denoise(val_data_dir=up_dir, val_filename=textfile, save_path = up_dir)

    # result_path = os.path.join("./results/denoise", filename)
    image_result = Image.open(pred_image)

    print("thanh cong !")
    # Ảnh gốc
    buffered_origin = io.BytesIO()
    image_origin = image
    image_origin.save(buffered_origin, format="PNG")
    img_origin_str = base64.b64encode(buffered_origin.getvalue()).decode("utf-8")
    print(img_origin_str)
    # Ảnh đã xử lý
    buffered_result = io.BytesIO()
    image_result.save(buffered_result, format="PNG")
    img_str = base64.b64encode(buffered_result.getvalue()).decode("utf-8")
    print(img_str)

In [4]:
denoise_url("https://prod-images-static.radiopaedia.org/images/55151890/LungLowDose__3.0__B31f_128_gallery.jpeg")

Seed:	19
cuda
Start Val


100%|██████████| 1/1 [00:00<00:00,  2.08it/s]

thanh cong !
...
